# Create a Graph from a text

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
- Define an Ontology
- Load a list of example text chunks. We will use the Lord of the Rings summary from this wikipedia page. 
- Create Graph using an Open source model using Groq APIs. 
- Save the graph to Neo4j db
- Visualise



Loading the graph maker functions ->

In [11]:
import os

In [12]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = "TEST"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "abc12345678"
os.environ["NEO4J_URI"]= "bolt://localhost:7687"

In [13]:
from knowledge_graph_maker import GraphMaker, Ontology, OpenAIClient
from knowledge_graph_maker import Document

# Define the Ontology. 

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```



For example lets use summaries of the LOTR books from the Wikipedia page. I have copied it into a file for easy import

In [14]:
from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list

Here is the ontology we will use for the LOTR summaries ->

In [15]:

ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)


## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [16]:

## Groq models
model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
oai_model="gpt-3.5-turbo"

## Use Groq
# llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)


## Create documents out of text chumks. 
Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 


In [17]:
import datetime
current_time = str(datetime.datetime.now())

In [18]:
def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary

In [19]:
example_text_list

['\nBilbo Baggins celebrates his birthday and leaves the Ring to Frodo, his heir. \nGandalf (a wizard) suspects it is a Ring of Power; seventeen years later, he confirms it was lost by the \nDark Lord Sauron and counsels Frodo to take it away from the Shire. Gandalf leaves, promising to return, \nbut fails to do so. Frodo sets out on foot with his cousin Pippin Took and gardener Sam Gamgee. \nThey are pursued by Black Riders, but meet some Elves, whose singing to Elbereth wards off the Riders. \nThe Hobbits take an evasive shortcut to Bucklebury Ferry, where they meet their friend Merry Brandybuck. \nMerry and Pippin reveal they know about the Ring and insist on joining Frodo on his journey.\n',
 "\nThey try to shake off the Black Riders by cutting through the Old Forest. Merry and Pippin are \ntrapped by the malign Old Man Willow, but are rescued by Tom Bombadil. Leaving Tom's house, \nthey are caught by a barrow-wight. Frodo, awakening from the barrow-wight's spell, calls Tom Bombadi

In [21]:
docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)

## Create Graph
Finally run the Graph Maker to generate graph. 

In [22]:
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

In [23]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  gpt-3.5-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-3.5-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-3.5-turbo


...

Using Model:  gpt-3.5-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-24 19:13:30 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Person",
            "name": "Sam"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Sam rescues Frodo from the tower of Cirith Ungol."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Frodo"
        },
        "node_2": {
            "label": "Place",
            "name": "Mordor"
        },
        "relationship": "Frodo and Sam set out across Mordor."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Frodo"
        },
        "node_2": {
            "label": "Place",
            "name": "Cracks of Doom"
        },
        "relationship": "Frodo reaches the edge of the Cracks of Doom."
    },
    {
        "node_1": {
            "labe

Using Model:  gpt-3.5-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-24 19:13:38 - INFO 
LLM Response:
```json
[
    {
        "node_1": {
            "label": "Person",
            "name": "Galadriel"
        },
        "node_2": {
            "label": "Person",
            "name": "Celeborn"
        },
        "relationship": "Galadriel and Celeborn say farewell to Treebeard."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Galadriel"
        },
        "node_2": {
            "label": "Person",
            "name": "Gandalf"
        },
        "relationship": "Galadriel says farewell to Gandalf."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Galadriel"
        },
        "node_2": {
            "label": "Person",
            "name": "Treebeard"
        },
        "relationship": "Galadriel says farewell to Treebeard."
    },
    {
        "node_1": {
   

Total number of Edges 151


In [9]:
for edge in graph:
    print(edge.model_dump(exclude=['metadata']), "\n\n")

{'node_1': {'label': 'Person', 'name': 'Bilbo Baggins'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Bilbo Baggins leaves the Ring to Frodo, his heir.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Gandalf counsels Frodo to take the Ring away from the Shire.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Sauron'}, 'relationship': 'Gandalf confirms the Ring was lost by the Dark Lord Sauron.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Gandalf counsels Frodo to take the Ring away from the Shire.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Frodo'}, 'node_2': {'label': 'Person', 'name': 'Pippin Took'}, 'relationship': 'Frodo sets out with his cousin Pippin Took.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Frodo'},

# Save the Graph to Neo4j 

In [10]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()


145